<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.63
    Uninstalling yfinance-0.2.63:
      Successfully uninstalled yfinance-0.2.63
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-23 14:14:59
-------------------
qualified stocks: 93
with latest results: 29
still star stocks: 17


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  35.47 K
Current:  1.32 C
-------------------
Today PnL: 27.22 K (0.21%)
Current PnL: -17.66 L (-12.31%)
CY Booked + Current PnL: -11.29 L (-7.87%)
-------------------
Total profit:  3.01 L
Total loss:  -20.67 L
-------------------
Total Booked + Current PnL: 15.69 L (13.46%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.84%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.06 C
Est FTT PnL: 74.13 L (56.36%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 8.32%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LICI,827.06,983.0,-5.02,H-LC,89.25,96974.0,11787.0,4277.0,0.53,13.84,4.41,18.85,35.0,2.76,0.74,29.11,XY25,NTT,INSURANCE
62,RELIANCE,1291.83,1526.0,6.87,X-LC,63.44,158998.0,18189.0,7330.0,-0.51,12.92,4.61,18.13,17.0,2.48,1.21,25.54,XY25,NTT,REFINERIES
89,VIPIND,488.80,489.0,-977.59,H-SC,98.92,83606.0,-11710.0,11747.0,4.88,-12.29,14.05,0.04,153.0,-1.00,0.64,64.90,OX40N,NTT,MISC
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,86.02,131139.0,-11340.0,11960.0,0.83,-7.96,9.12,0.44,242.0,-0.95,1.00,54.78,OX40N,NTT,BANKS
78,TATAELXSI,7332.28,7332.0,-15.48,X-MC,61.29,88704.0,-13948.0,13944.0,0.27,-13.59,15.72,-0.00,57.0,-1.00,0.68,34.84,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LALPATHLAB,2804.19,3545.00,-32.88,H-SC,43.01,154572.0,342.0,40405.0,-3.50,0.22,26.14,26.42,126.0,0.01,1.18,21.89,X40N,NTT,HEALTHCARE
34,HEROMOTOCO,4311.81,6039.03,-1.90,H-MC,44.09,149426.0,-1487.0,61937.0,-1.59,-0.99,41.45,40.06,100.0,-0.02,1.14,22.22,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-37.21,H-SC,65.59,99778.0,-726.0,31370.0,2.32,-0.72,31.44,30.49,97.0,-0.02,0.76,31.45,X5K,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,83.87,68698.0,-44851.0,84904.0,7.42,-39.50,123.59,35.27,260.0,-0.53,0.52,68.14,XR,NTT,HOTELS
89,VIPIND,488.80,489.00,-977.59,H-SC,98.92,83606.0,-11710.0,11747.0,4.88,-12.29,14.05,0.04,153.0,-1.00,0.64,64.90,OX40N,NTT,MISC
26,DIXON,14367.00,18940.15,-11.92,H-LC,27.96,116360.0,1424.0,35164.0,3.55,1.24,30.22,31.83,101.0,0.04,0.89,33.36,X40N,ATH,IT
4,ALKYLAMINE,2347.98,4567.52,-1.33,X-SC,97.85,91912.0,-9051.0,104495.0,3.46,-8.96,113.69,94.53,60.0,-0.09,0.70,36.27,SR,ATH,CHEMICALS
66,SAMMAANCAP,170.35,326.00,-160.64,M-SC,26.88,73734.0,-28476.0,121868.0,3.37,-27.86,165.28,91.37,200.0,-0.23,0.56,19.69,XY25,NTT,FINANCE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LALPATHLAB,2804.19,3545.00,-32.88,H-SC,43.01,154572.0,342.0,40405.0,-3.50,0.22,26.14,26.42,126.0,0.01,1.18,21.89,X40N,NTT,HEALTHCARE
71,SIEMENS,4406.90,7969.85,-14.29,X-LC,18.28,77560.0,-32612.0,121684.0,-3.48,-29.60,156.89,80.85,11.0,-0.27,0.59,14.23,AR,ATH,ELECTRICAL
51,LAOPALA,369.40,464.00,121.82,H-SC,78.49,70434.0,-30412.0,56235.0,-2.32,-30.16,79.84,25.61,131.0,-0.54,0.54,27.03,AR,NTT,CERAMICS
44,INFY,1461.46,2275.00,-11.34,X-LC,66.67,284324.0,22723.0,122913.0,-2.13,8.69,43.23,55.67,1.0,0.18,2.17,15.32,X40,BTT,IT
17,CAMPUS,294.86,393.00,NaN,NaN,56.99,158353.0,-9422.0,65257.0,-1.92,-5.62,41.21,33.28,201.0,-0.14,1.21,22.60,XY24,NTT,FOOTWEAR


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,UJJIVANSFB,52.77,53.0,51.69,M-SC,86.02,131139.0,-11340.0,11960.0,0.83,-7.96,9.12,0.44,242.0,-0.95,1.00,54.78,OX40N,NTT,BANKS
89,VIPIND,488.80,489.0,-977.59,H-SC,98.92,83606.0,-11710.0,11747.0,4.88,-12.29,14.05,0.04,153.0,-1.00,0.64,64.90,OX40N,NTT,MISC
78,TATAELXSI,7332.28,7332.0,-15.48,X-MC,61.29,88704.0,-13948.0,13944.0,0.27,-13.59,15.72,-0.00,57.0,-1.00,0.68,34.84,OX40N,NTT,IT
76,SYMPHONY,1306.42,1306.0,-34.31,M-SC,1.08,140170.0,-30971.0,30922.0,-0.95,-18.10,22.06,-0.03,190.0,-1.00,1.07,0.74,OX40N,NTT,DURABLES
42,INDIGOPNTS,1407.73,1408.0,118.19,H-SC,92.47,142922.0,-31637.0,31672.0,2.82,-18.12,22.16,0.02,127.0,-1.00,1.09,23.94,OX40N,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,INFY,1461.46,2275.00,-11.34,X-LC,66.67,284324.0,22723.0,122913.0,-2.13,8.69,43.23,55.67,1.0,0.18,2.17,15.32,X40,BTT,IT
80,TCS,3794.03,4998.00,-18.73,X-LC,33.33,264849.0,-31085.0,124982.0,-1.17,-10.50,47.19,31.73,2.0,-0.25,2.02,6.01,X40,BTT,IT
36,HINDUNILVR,2413.81,2723.00,-18.73,X-LC,34.41,255125.0,-15222.0,49851.0,-0.19,-5.63,19.54,12.81,5.0,-0.31,1.94,6.70,XY25,NTT,FMCG
15,BRITANNIA,4983.33,6446.05,11.96,X-LC,59.14,256703.0,27470.0,39815.0,-0.12,11.98,15.51,29.35,7.0,0.69,1.96,21.97,XY25,ATH,FMCG
57,NESTLEIND,2268.60,2755.00,-13.10,X-LC,36.56,276026.0,10600.0,46317.0,-0.05,3.99,16.78,21.44,10.0,0.23,2.10,10.44,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ADANIPORTS,1103.69,1583.0,-4.03,M-LC,35.48,96312.0,17950.0,16084.0,0.53,22.91,16.7,43.43,77.0,1.12,0.73,29.44,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,5.33,H-SC,88.17,118507.0,16503.0,28525.0,1.56,16.18,24.07,44.14,125.0,0.58,0.90,35.05,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,-4.03,M-LC,35.48,96312.0,17950.0,16084.0,0.53,22.91,16.70,43.43,77.0,1.12,0.73,29.44,XY24,NTT,MISC
92,WIPRO,243.46,420.00,-4.72,M-LC,82.80,163314.0,12369.0,97090.0,-1.17,8.19,59.45,72.51,68.0,0.13,1.24,12.50,XR,NTT,IT
85,UNIONBANK,123.87,163.00,-6.44,M-LC,64.52,163376.0,22536.0,21958.0,0.15,16.00,13.44,31.59,86.0,1.03,1.25,41.92,XY24,NTT,BANKS
39,ICICIGI,1839.64,2260.25,-11.32,H-MC,74.19,145854.0,9721.0,21411.0,2.26,7.14,14.68,22.86,34.0,0.45,1.11,20.65,X40,ATH,INSURANCE


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WHIRLPOOL,1167.49,2270.00,-30.25,M-SC,90.32,203374.0,31753.0,130322.0,2.47,18.50,64.08,94.43,194.0,0.24,1.55,50.10,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-4.72,M-LC,82.80,163314.0,12369.0,97090.0,-1.17,8.19,59.45,72.51,68.0,0.13,1.24,12.50,XR,NTT,IT
41,INDIAMART,2327.09,4871.06,-48.48,H-SC,84.95,133979.0,10643.0,124185.0,1.51,8.63,92.69,109.32,122.0,0.09,1.02,32.36,AR,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-1.90,H-MC,44.09,149426.0,-1487.0,61937.0,-1.59,-0.99,41.45,40.06,100.0,-0.02,1.14,22.22,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-37.21,H-SC,65.59,99778.0,-726.0,31370.0,2.32,-0.72,31.44,30.49,97.0,-0.02,0.76,31.45,X5K,ATH,METALS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-53.72,L-SC,83.87,68698.0,-44851.0,84904.0,7.42,-39.50,123.59,35.27,260.0,-0.53,0.52,68.14,XR,NTT,HOTELS
7,ASIANTILES,75.41,137.00,7383.33,M-SC,100.00,82104.0,-11706.0,88327.0,0.59,-12.48,107.58,81.67,238.0,-0.13,0.63,59.00,XR,NTT,CERAMICS
54,MASFIN,326.60,402.39,-23.22,H-SC,51.61,87105.0,-10875.0,33614.0,0.16,-11.10,38.59,23.21,141.0,-0.32,0.66,26.33,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4567.52,-1.33,X-SC,97.85,91912.0,-9051.0,104495.0,3.46,-8.96,113.69,94.53,60.0,-0.09,0.70,36.27,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,-4.03,M-LC,35.48,96312.0,17950.0,16084.0,0.53,22.91,16.70,43.43,77.0,1.12,0.73,29.44,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,75.41,137.00,7383.33,M-SC,100.00,82104.0,-11706.0,88327.0,0.59,-12.48,107.58,81.67,238.0,-0.13,0.63,59.00,XR,NTT,CERAMICS
89,VIPIND,488.80,489.00,-977.59,H-SC,98.92,83606.0,-11710.0,11747.0,4.88,-12.29,14.05,0.04,153.0,-1.00,0.64,64.90,OX40N,NTT,MISC
4,ALKYLAMINE,2347.98,4567.52,-1.33,X-SC,97.85,91912.0,-9051.0,104495.0,3.46,-8.96,113.69,94.53,60.0,-0.09,0.70,36.27,SR,ATH,CHEMICALS
69,SFL,1016.35,1287.00,326.06,H-SC,96.77,192365.0,-69853.0,139676.0,0.49,-26.64,72.61,26.63,145.0,-0.50,1.47,19.60,XY24,NTT,MISC
75,SURYODAY,189.78,240.00,57.89,H-SC,95.16,111904.0,-38022.0,77695.0,1.14,-25.36,69.43,26.46,143.0,-0.49,0.85,43.36,XR,NTT,BANKS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.29
1,25,42.73
2,50,71.36


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.5
LC    38.1
MC    21.7
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.09
XY25     14.14
X40      12.51
X40N     10.81
XR       10.31
OX40N     8.33
AR        7.91
X5K       2.25
X200      1.47
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.41
X-LC    15.46
M-SC    13.97
H-LC    13.29
H-MC     8.52
M-LC     8.25
X-MC     7.62
M-MC     5.56
X-SC     2.74
L-SC     2.38
L-LC     1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.48,-2.97,37.42
IT,12.11,-11.90,67.47
BANKS,8.00,-8.70,53.06
FINANCE,7.67,-19.74,63.09
MISC,6.72,-22.62,70.57
PAINTS,5.61,-18.62,41.99
ELECTRICAL,5.09,-5.27,50.87
HEALTHCARE,4.65,-5.77,36.67
AUTO,4.44,-16.07,67.11


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2852205.0,25
XR,1105506.0,13
AR,1033384.0,9
X40,679627.0,9
XY25,530996.0,10
X40N,512648.0,10
OX40N,338838.0,11
SR,188055.0,2
X5K,104539.0,3


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1951509.0,19
M-SC,1495540.0,18
X-LC,634873.0,9
M-MC,613731.0,5
H-LC,613611.0,12
H-MC,506598.0,7
X-MC,418555.0,6
M-LC,361016.0,6
L-SC,284172.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,905504.0,6
M-SC,XY24,605354.0,5
H-SC,AR,523812.0,3
M-MC,XY24,404914.0,3
M-SC,XR,338467.0,4
X-LC,X40,326276.0,3
M-LC,XY24,245158.0,4
H-LC,X40N,240185.0,5
H-MC,XY24,239862.0,2


In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
